### Smoothing exercise
create universe of stocks of QTradabelStocksUS from the following website:

https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus

In [9]:
# Imports
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import time

from quantopian.pipeline.experimental import QTradableStocksUS

### Pipeline construction using QTradableStocksUS as Screen

In [12]:
# function used to create a pipeline with the QTradableStocksUS as a screen. 
# The pipeline contains common metrics as columns.

def make_pipeline():
    
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    
    return Pipeline(
        columns={
            'AverageDollarVolume200': average_day_dv_200,
            'MarketCap': market_cap,
            'Price': price,
            'Volume': volume,
            'Sector': sector,
        },
        screen = QTradableStocksUS()
    )

In [14]:
# Pipeline is run over this time range and outputs a dataframe indexed by asset name:
START_DATE = '2017'
END_DATE = '2019-07-29'

start = time.time()
QTU_pipeline = run_pipeline(make_pipeline(), START_DATE, END_DATE, chunksize=252)
print 'Took %s seconds' % (time.time() - start)

Took 49.0925941467 seconds


In [16]:
QTU_pipeline.head(2)

AverageDollarVolume200  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])             1.736107e+08   
                          Equity(24 [AAPL])            3.048351e+09   

                                                MarketCap   Price  Sector  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])   8.129387e+09   18.55     101   
                          Equity(24 [AAPL])  6.175885e+11  115.84     311   

                                                 Volume  
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])    3871123.0  
                          Equity(24 [AAPL])  23251752.0

### Lets add factor of 1 year returns, demeaned by sector, and then rank

In [20]:
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.factors import Returns
from zipline.pipeline.factors import DailyReturns

In [21]:


def make_pipeline():
    
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    
    # New factors
    # create a factor of 1 year returns, demeaned by sector, rank, zscore
    factor = (
        Returns(window_length = 252).\
        demean(groupby = Sector()).\
        rank().\
        zscore()
    
    )
    
    # Use the newly created factor as an input into SimpleMovingAverage, with a window length of 5
    # Also rank and zscore (don't need to demean by sector)
    factor_smoothed = (
                    SimpleMovingAverage(inputs = [factor], window_length=5).\
                    rank().\
                    zscore()
    )
    
    
    return Pipeline(
        columns={
            'AverageDollarVolume200': average_day_dv_200,
            'MarketCap': market_cap,
            'Price': price,
            'Volume': volume,
            'Sector': sector,
            '1yrReturns': factor,
            '5dAvgReturns': factor_smoothed
        },
        screen = QTradableStocksUS()
    )

In [22]:
# Pipeline is run over this time range and outputs a dataframe indexed by asset name:
START_DATE = '2017'
END_DATE = '2019-07-29'

start = time.time()
QTU_pipeline2 = run_pipeline(make_pipeline(), START_DATE, END_DATE, chunksize=252)

In [23]:
QTU_pipeline2

1yrReturns  5dAvgReturns  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       -1.663162     -1.694499   
                          Equity(24 [AAPL])       0.242636      0.270176   
                          Equity(31 [ABAX])       0.731982      0.638386   
                          Equity(41 [ARCB])       0.571808      0.623764   
                          Equity(52 [ABM])        1.027217      0.888291   
                          Equity(53 [ABMD])       1.407291      1.366831   
                          Equity(62 [ABT])        0.527013      0.414403   
                          Equity(64 [GOLD])       1.425616      1.415350   
                          Equity(67 [ADSK])       0.628819      0.669624   
                          Equity(76 [TAP])       -0.091286     -0.100028   
                          Equity(84 [ACET])       0.448284      0.184437   
                          Equity(110 [RAMP])      0.945773      0.826479   
                          Equity(114 [ADBE])      0.229062      0.211688   
                          Equity(122 [ADI])       1.038755      1.035176   
                          Equity(128 [ADM])       0.805281      0.791918   
                          Equity(154 [AEM])       0.225669      0.114650   
                          Equity(161 [AEP])      -0.201914     -0.248908   
                          Equity(166 [AES])       0.459821      0.496153   
                          Equity(168 [AET])       1.160243      1.221940   
                          Equity(185 [AFL])       0.063459      0.035558   
                          Equity(197 [AGCO])      0.372269      0.357908   
                          Equity(216 [HES])      -1.352995     -0.460928   
                          Equity(239 [AIG])      -0.360052     -0.421714   
                          Equity(247 [AIN])       0.924055      0.876327   
                          Equity(253 [AIR])       0.368875      0.425702   
                          Equity(266 [AJG])       0.560948      0.520080   
                          Equity(270 [AKRX])     -0.863650     -0.944121   
                          Equity(289 [MATX])     -1.105947     -1.241879   
                          Equity(300 [ALK])      -0.234492     -0.372531   
                          Equity(301 [ALKS])     -0.180196     -0.319359   
...                                                    ...           ...   
2019-07-29 00:00:00+00:00 Equity(52186 [AFIN])   -0.619640     -0.625765   
                          Equity(52209 [ALLK])   -0.174803     -0.016795   
                          Equity(52211 [TLRY])    1.602626      1.597381   
                          Equity(52233 [BE])     -1.569391     -1.599869   
                          Equity(52241 [TENB])   -0.089159     -0.085219   
                          Equity(52242 [FOCS])   -0.849088     -0.864626   
                          Equity(52244 [BRY])     0.819688      0.844721   
                          Equity(52288 [CWK])          NaN           NaN   
                          Equity(52291 [SONO])         NaN           NaN   
                          Equity(52361 [BPR])          NaN           NaN   
                          Equity(52407 [FTDR])         NaN           NaN   
                          Equity(52408 [GTX])          NaN           NaN   
                          Equity(52424 [ELAN])         NaN           NaN   
                          Equity(52425 [FTCH])         NaN           NaN   
                          Equity(52427 [EB])           NaN           NaN   
                          Equity(52446 [UPWK])         NaN           NaN   
                          Equity(52450 [SVMK])         NaN           NaN   
                          Equity(52491 [GH])           NaN           NaN   
                          Equity(52494 [ESTC])         NaN           NaN   
                          Equity(52508 [ALLO])         NaN           NaN   
                          Equity(52511 [LTHM])         NaN           NaN   
                    